In [75]:
# import package
from tqdm import tqdm
import numpy as np
import random
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import string
from torch.utils.data import Dataset, DataLoader
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
nltk.download('stopwords')
nltk.download("wordnet")
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [76]:
# ! unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

In [77]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [78]:
# setting seed
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(42)
MAX_LENGTH = 30
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [79]:
# import dataset
train= pd.read_csv('./train.csv')
test= pd.read_csv("./test.csv")

In [80]:
# data cleaning
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', string)

def clean_sentence(sentence):
    # remove URLS 
    sentence = re.sub(r'http\S+', ' ', sentence)
    # remove emoji's
    sentence = remove_emoji(sentence)
    # remove punctuation
    sentence = re.sub("[^0-9A-Za-z ]", "" , sentence)
    # remove double spaces
    sentence = sentence.replace('  ',"")
    
    return sentence.strip()

def remove_stopwords(tokens, stopwords):
    clean_token = [word for word in tokens if word not in stopwords]
    return clean_token

# lemmalization 
def lemmatize(tokens, lemma):
    lemmatized_tokens = [lemma.lemmatize(token, pos = 'v') for token in tokens]
    return lemmatized_tokens

In [81]:
# fix sentence length
def trunc_padding(sentence):
    modify_sentence = sentence.copy()
    if len(modify_sentence) >= MAX_LENGTH:
        modify_sentence = modify_sentence[:MAX_LENGTH]
    else:
        modify_sentence.extend(list(["0"] * (MAX_LENGTH - len(modify_sentence))))
    return modify_sentence

In [82]:
# processing data cleaning
stopwords = nltk.corpus.stopwords.words('english')
lemma = WordNetLemmatizer()

def processing(df, stopwords, lemma):
    df['text'] = df['text'].apply(lambda sentence: sentence.lower())
    df['text'] = df['text'].apply(lambda sentence: clean_sentence(sentence))
    # tokenization
    df['text'] = df['text'].apply(lambda sentence: sentence.split())
    # remove stopwords
    df['text'] = df['text'].apply(lambda sentence: remove_stopwords(sentence, stopwords))
    # lemmalization 
    df['text'] = df['text'].apply(lambda sentence: lemmatize(sentence, lemma))
    # sentence length before padding
    df['length'] = df['text'].apply(lambda x: len(x))
    # fix sentence length
    df['text'] = df['text'].apply(lambda sentence: trunc_padding(sentence))
    # sentence length after padding
    df['length_padding'] = df['text'].apply(lambda x: len(x))
    
processing(train, stopwords, lemma)
processing(test, stopwords, lemma)

In [83]:
train.head()

,id,keyword,location,text,target,length,length_padding
0,1,NaN,NaN,"[deeds, reason, earthquake, may, allah, forgiv...",1,7,30
1,4,NaN,NaN,"[forest, fire, near, la, ronge, sask, canada, ...",1,7,30
2,5,NaN,NaN,"[residents, ask, shelter, place, notify, offic...",1,11,30
3,6,NaN,NaN,"[13000, people, receive, wildfires, evacuation...",1,7,30
4,7,NaN,NaN,"[get, send, photo, ruby, alaska, smoke, wildfi...",1,9,30


In [84]:
test.head()

,id,keyword,location,text,length,length_padding
0,0,NaN,NaN,"[happen, terrible, car, crash, 0, 0, 0, 0, 0, ...",4,30
1,2,NaN,NaN,"[hear, earthquake, different, cities, stay, sa...",7,30
2,3,NaN,NaN,"[forest, fire, spot, pond, geese, flee, across...",10,30
3,9,NaN,NaN,"[apocalypse, light, spokane, wildfires, 0, 0, ...",4,30
4,11,NaN,NaN,"[typhoon, soudelor, kill, 28, china, taiwan, 0...",6,30


## Assign unique index to each word, used for word embedding

In [85]:
# get all vocabulary
vocab_list = []
for sentence in train['text']:
    vocab_list.append(sentence)
vocab = build_vocab_from_iterator(vocab_list, specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

### Dataset and DataLoader

In [86]:
class TweetDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y 
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        sentence = self.x[idx]
        x = torch.tensor(vocab(sentence), dtype=torch.long).to(device)
        y = torch.tensor(self.y[idx], dtype=torch.long).to(device)
        return x, y

In [87]:
# model configuration
model_config = {
    'vocab_size': len(vocab),
    'hidden_dim' : 256,
    'embedding_dim' : 200,
    'num_classes' : 2,
    'n_layers': 2,
    'dropout': 0.2
}

In [88]:
train_data = TweetDataset(train['text'], train['target'])
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

In [89]:
# batch test
for batch in train_dataloader:
    print(batch)
    break

[tensor([[  191,  1122,  2990,  ...,     1,     1,     1],
        [    1,   274, 11380,  ...,     1,     1,     1],
        [   54,   518,  4206,  ...,     1,     1,     1],
        ...,
        [ 4056,   605,  1295,  ...,     1,     1,     1],
        [    3,    67,    24,  ...,     1,     1,     1],
        [  292,   483,  3151,  ...,     1,     1,     1]], device='cuda:0'), tensor([1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
        0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0], device='cuda:0')]


# Model

# GRU

In [90]:
# GRU
class GRU(nn.Module):
    def __init__(self, config):
        super(GRU, self).__init__()
        self.config = config
        self.embedding = nn.Embedding(config['vocab_size'], config['embedding_dim'])
        self.gru = nn.GRU(config['embedding_dim'], config['hidden_dim'], bidirectional=True)
        self.fc = nn.Linear(config['hidden_dim'] * 2, config['num_classes'])
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, X):
        embeds = self.embedding(X).permute(1, 0, 2)
        hidden_state = Variable(torch.zeros(2, len(X), self.config['hidden_dim'])).to(device)
        cell_state = Variable(torch.zeros(2, len(X), self.config['hidden_dim'])).to(device)
        output, (final_hidden_state, final_cell_state) = self.gru(embeds, hidden_state)
        output = output.permute(1, 0, 2)
        output = self.fc(output[:, -1, :])
        return self.sigmoid(output)


In [91]:
# calculate accuracy
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    prediction = preds.argmax(dim=1)
    correct = (prediction == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

# Training

In [92]:
# make prediction
model = GRU(model_config)
model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [93]:
# training model
epoch_loss = 0
epoch_acc = 0
for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        inputs, target = batch
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        epoch_loss += loss
        
        acc = binary_accuracy(outputs, target)
        epoch_acc += acc.item()
    
    print("The training loss at epoch {} is {}; The training accuracy is {}".format(epoch, epoch_loss / len(train_dataloader), 
                                                                                    round(epoch_acc/len(train_dataloader), 3)))
    epoch_loss = 0
    epoch_acc = 0


100%|██████████| 119/119 [00:01<00:00, 76.91it/s]


The training loss at epoch 0 is 0.670957624912262; The training accuracy is 0.586


100%|██████████| 119/119 [00:01<00:00, 100.60it/s]


The training loss at epoch 1 is 0.5604051947593689; The training accuracy is 0.745


100%|██████████| 119/119 [00:01<00:00, 100.72it/s]


...

The training loss at epoch 7 is 0.39321064949035645; The training accuracy is 0.92


100%|██████████| 119/119 [00:02<00:00, 45.51it/s]


The training loss at epoch 8 is 0.3868984580039978; The training accuracy is 0.926


100%|██████████| 119/119 [00:03<00:00, 31.21it/s]

The training loss at epoch 9 is 0.3875861167907715; The training accuracy is 0.925


In [94]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [95]:
# make prediction
def generate_prediction(inputs, model):
    inputs = torch.tensor(vocab(inputs), dtype=torch.long).to(device).unsqueeze(0)
    outpus =  model(inputs).argmax(dim=1).item()
    return outpus

test['target'] = test['text'].apply(lambda sentence: generate_prediction(sentence, model))

In [96]:
# store result
submission = test[['id', 'target']]
submission.to_csv('submission_gru.csv', index=False)
print(sum(test['target']))

1453


In [97]:
# calculate test accuracy
import pandas as pd
# Load the target and submission dataframes
target_df = pd.read_csv('./check.csv')
submission_df = pd.read_csv('./submission_gru.csv')
# Merge the dataframes on the 'id' column
merged_df = pd.merge(target_df, submission_df, on='id')
# Calculate the accuracy rate
accuracy_rate = (merged_df['target_x'] == merged_df['target_y']).mean()
print('Accuracy rate:', accuracy_rate)

Accuracy rate: 0.7446351931330472
